# Neural Network for Airbnb 

**Outline**

* [Read Data](#read)
* [Data Transformation](#transform)
* [Model Building](#model) 
    * [Parameter Tuning](#tune)
* [Important Features](#feature) 
* [Reference](#refer)

In [28]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import model_selection
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import scale

from sklearn.metrics import classification_report

SEED = 12345

---

## <a id="read">Read Data</a>

In [6]:
def data_reader():
    """
    read data into notebook 
    """
        
    data_dir = os.path.join('.') #, 'data'

    train_binary_path = os.path.join(data_dir, 'train_session_updated.csv')  
    train_binary = pd.read_csv(train_binary_path) #, index_col = 0
    if 'Unnamed: 0' in train_binary.columns:
        del train_binary['Unnamed: 0']
    drop_feature = ['user_id', 'total_secs_elapsed', 'date_account_created','timestamp_first_active','date_first_booking','country_destination']
    train_binary.drop(drop_feature, axis=1, inplace=True)

    return train_binary #

In [7]:
train_binary = data_reader()

In [8]:
train_binary.columns

Index(['ajax_refresh_subtotal', 'dashboard', 'edit', 'header_userpic',
       'personalize', 'similar_listings', 'total_actions', 'obs_count',
       'unique_action', 'unique_device', 'avg_time', 'gender', 'age',
       'signup_method', 'signup_flow', 'language', 'affiliate_channel',
       'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
       'first_device_type', 'first_browser', 'isNDF'],
      dtype='object')

## <a id="transform">Data Transformation</a>

In [9]:
categorical = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel',
                'affiliate_provider','first_affiliate_tracked','signup_app','first_device_type',
               'first_browser']    

In [16]:
# Convert data type as 'category'
for i in categorical:
    train_binary[i] = train_binary[i].astype('category')

In [17]:
# Create dummy variables
train_binary_dummy = pd.get_dummies(train_binary, columns = categorical)
train_binary_dummy.head()

,ajax_refresh_subtotal,dashboard,edit,header_userpic,personalize,similar_listings,total_actions,obs_count,unique_action,unique_device,...,first_browser_RockMelt,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_Sogou Explorer,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,2,4,0,2,4,3,15,3.713572,13,2,...,0,1,0,0,0,0,0,0,0,0
1,0,2,0,2,0,0,4,6.672033,25,2,...,0,0,0,0,0,0,0,0,0,0
2,21,3,0,2,26,12,64,6.194405,20,1,...,0,1,0,0,0,0,0,0,0,0
3,12,2,18,2,12,17,63,5.181784,38,2,...,0,0,0,0,0,0,0,0,0,0
4,1,4,0,1,4,1,11,3.295837,11,2,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Split data into response and predictors
y = train_binary_dummy['isNDF']
x = train_binary_dummy.drop('isNDF', axis=1)

In [34]:
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

In [37]:
x = my_scaler(x)

## <a id="model">Model Building</a>

In [39]:
def fit_nn(x, y, param_setting={}, fold=5, seed=SEED):
    """Neural Network for Classification, get the CV AUC"""        
    
   # set seed and default parameter 
    params_default = {'random_state':seed,
                     'activation': 'logistic',
                     }

    # update the input parameters
    params = dict(params_default)
    params.update(param_setting)
    
    seed= SEED        
    kfold = model_selection.KFold(n_splits=fold, random_state=seed)
    model = MLPClassifier(**params)    
    
    results = model_selection.cross_val_score(model, x, y, cv=kfold, scoring='roc_auc')
    print(results.mean())
    
    model.fit(x, y)
    
    return model

In [44]:
param_setting={'max_iter':2000, 'early_stopping':True, 'learning_rate_init':0.01}
nn_base = fit_nn(x, y, fold=5, param_setting=param_setting, seed=SEED)

0.7391501297209147


In [48]:
nn_base

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=12345,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

### <a id="tune">Parameter Tuning </a>

**Steps**
1. Use logistics activation function for 1 hidden layer neural network
2. Tune the model using combination of node in the hidden layer and alpha and pick the best parameters.

**Key parameters**

* **hidden_layer_sizes**: The ith element represents the number of neurons in the ith hidden layer. length = n_layers - 2. Default is (100,), which means that there is only 1 hidden layer with 100 nodes, since len((100,))=1=3-2. 3 means that there are 3 layers including input and output layer. For architecture 56:25:11:7:5:3:1 with input 56 and 1 output hidden layers will be (25:11:7:5:3). So tuple hidden_layer_sizes = (25,11,7,5,3,)
* **alpha**: L2 penalty (regularization term) parameter.

In [49]:
def parameter_tuning(model, X_train, y_train, param_grid, fold=5):   
    """
    Tune a tree based model using GridSearch, and return a model object with an updated parameters
    
    Parameters
    ----------
    model: sklearn's ensemble tree model
        the model we want to do the hyperparameter tuning.
    
    X_train: pandas DataFrame
        Preprocessed training data. Note that all the columns should be in numeric format.
    
    y_train: pandas Series
    
    param_grid: dict
        contains all the parameters that we want to tune for the responding model.    
        

    Note
    ----------
    * we use kfold in GridSearchCV in order to make sure the CV Score is consistent with the score 
      that we get from all the other function, including fit_bagging, fit_randomforest and fit_gbm. 
    * We use model_selection.KFold with fixed seed in order to make sure GridSearchCV uses the same seed as model_selection.cross_val_score.
    
    """
    seed=SEED
    
#     if 'n_estimators' in param_grid:
#         model.set_params(warm_start=True) 
    
    kfold = model_selection.KFold(n_splits=fold, random_state=seed)
    gs_model = GridSearchCV(model, param_grid, cv=kfold, scoring='roc_auc')
    gs_model.fit(X_train, y_train)
    
    # best hyperparameter setting
    print('best parameters:{}'.format(gs_model.best_params_)) 
    print('best score:{}'.format(gs_model.best_score_)) 
    
    # refit model on best parameters
    model.set_params(**gs_model.best_params_)
    model.fit(X_train, y_train)

    return(model)

In [50]:
params = {
    'max_iter':2000, 
    'early_stopping':True, 
    'learning_rate_init':0.01,
    'random_state':SEED
}

nn = MLPClassifier(**params)

In [51]:
param_grid_nn_1 = {
    'hidden_layer_sizes': [(50,) ,(100,), (150,), (200,)],
    'alpha': [0.001, 0.01, 0.1, 1, 10],
}

In [52]:
nn_2 = parameter_tuning(nn, x, y, param_grid_nn_1)

best parameters:{'hidden_layer_sizes': (150,), 'alpha': 0.01}
best score:0.7425422581662595


## <a id="feature">Important Feature</a>

**[To do]** Partial Dependence Plots

## <a id="refer">Reference</a>

* [sklearn MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)
* [sklearn scale](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html)
* [Stackoverflow: hidden_layer_sizes](https://stackoverflow.com/questions/35363530/python-scikit-learn-mlpclassifier-hidden-layer-sizes)
* [partial dependence plots](http://scikit-learn.org/stable/auto_examples/ensemble/plot_partial_dependence.html)